![logo](https://img-prod-cms-rt-microsoft-com.akamaized.net/cms/api/am/imageFileData/RE1Mu3b?ver=5c31)

# **Fabric**
### Simulating streaming data for Realtime Analytics ⚡ using fabric Data Engineering notebook
### [Fabric RTA in a Day](https://aka.ms/fabricrtainaday) - Lab3 🪵
This notebook is used to take a CSV file with Log Data and stream it to Eventstreams as a simulated CustomApp source. 

The notebook will stream one row at at time (roughly 1 row/sec) into the CustomApp endpoint of the Eventstream. Eventstream will automatically ingest to the KQL database.

##### Please set your `ConnectionString` value **in the first code cell**. 
`insert screenshot create new eventstream`

##### 1st - sets variables

In [6]:
ConnectionString = ''

#SampleCsv = 'https://adxsamplefiles.blob.core.windows.net/publiccsvsamples/logsbenchmark-onegb/2014/03/08/13/data.csv.gz' #222KB, 7530 rows
SampleCsv = 'https://adxsamplefiles.blob.core.windows.net/publiccsvsamples/logsbenchmark-onegb/2014/03/08/00/data.csv.gz' #7MB, 254424 rows

StatementMeta(, 1e19c6c7-614f-434a-80ca-9635de97a643, 7, Finished, Available)

##### 2nd - installs required library

In [7]:
pip install azure-eventhub==5.11.5 --upgrade --force --quiet

StatementMeta(, 1e19c6c7-614f-434a-80ca-9635de97a643, 8, Finished, Available)

Note: you may need to restart the kernel to use updated packages.


##### 3rd - imports required libraries

In [16]:
from pyspark import SparkFiles
from pyspark.sql.functions import col
import time
import os
import datetime
import json
from azure.eventhub import EventHubProducerClient, EventData

StatementMeta(, 1e19c6c7-614f-434a-80ca-9635de97a643, 17, Finished, Available)

##### 4th - sends the data

In [17]:
spark.sparkContext.addFile(SampleCsv)
df = spark.read.options(header="false").csv("file://" + SparkFiles.get("data.csv.gz"))

df = df.select(
    col("_c0").alias("Timestamp"),
    col("_c1").alias("Source"),
    col("_c2").alias("Node"),
    col("_c3").alias("Level"),
    col("_c4").alias("Component"),
    col("_c5").alias("ClientRequestId"),
    col("_c6").alias("Message"),
    col("_c7").alias("Properties") 
)#.take(1)

# display(df.take(10))
# df.toJSON().collect()[0]

StatementMeta(, 1e19c6c7-614f-434a-80ca-9635de97a643, 18, Finished, Available)

In [18]:
producer = EventHubProducerClient.from_connection_string(conn_str=ConnectionString)

z = df.count() 
x = 0
y = 1

for x in range(0, z): 
    b = producer.create_batch() 
    j = df.toJSON().collect()[x:y][0]
    # print(j)
    b.add(EventData(j))
    producer.send_batch(b) # send it!
    time.sleep(1)
    producer.close()
    x=y
    y=y+1

StatementMeta(, 1e19c6c7-614f-434a-80ca-9635de97a643, 19, Submitted, Running)